# Erik's run script for the $Z_2$ gauge theory with Fermions

**Authors:**
- Clement Charles
- Erik Gustafson
- Elizabeth Hardt
- Florian Herren
- Norman Hogan
- Henry Lamm
- Sara Starecheski
- Ruth Van der Water
- Mike Wagman

The following code brings together all the necessary algorithms together to the simulations for our production run.

In [51]:
#Standard modules
import sys
import numpy as np
import yaml
import pandas
#Qiskit modules
from qiskit import IBMQ, QuantumCircuit, transpile, QuantumRegister, execute
from qiskit.providers.ibmq import RunnerResult
from qiskit import Aer
from qiskit.providers import aer
from qiskit.providers.aer import noise
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info import Statevector
# from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
from qiskit.visualization import plot_histogram

import datetime
#Project modules
sys.path.append('./modules/')
sys.path.append('./inputs/')
sys.path.append('./plots/')
import StatePrep as sprep
import DD_PassManager as dd_pm
import paulitwirlingpass as pts
from Z2analysis import nsites2nqubits
import Trotterization as Trot

# for plotting
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from IPython.display import Image
plt.rcParams["figure.autolayout"] = True
colors = list(mcolors.TABLEAU_COLORS.keys())
%matplotlib inline

#For testing reload modules after edits
from importlib import reload
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Get input parameters from `yaml` file

In [52]:
### get run parameters from YAML input file ###

stream = open('inputs/Z2_input.yaml', 'r')
dict_in = yaml.safe_load(stream)
stream.close()

account_info = dict_in['account_info'] # dictionary of account information
backend_opts = dict_in['backend_opts']   # dictionary of machine/simulator choices
simulation_opts = dict_in['simulation_opts'] # dictionary of lattice-simulation parameters
runtime_opts = dict_in['runtime_opts'] # dictionary of runtime parameters

### transpilation parameters ###
initial_layout = simulation_opts['layout']['initial_layout']
### Lattice parameters ###
mass = simulation_opts['lattice_params']['mass']       #fermion mass in lattice-spacing units
nsites = simulation_opts['lattice_params']['nsites']   #must be even for staggered simulation
nqubits = nsites2nqubits(nsites)
print(nqubits, nsites)
### Time evolution ###
tf = simulation_opts['time_evolution']['tf']       # total time interval in lattice-spacing units
ntrotter = simulation_opts['time_evolution']['ntrotter'] # number of trotter steps
epsilon = tf/ntrotter                              # step size in lattice-spacing units

### Pauli Twirling ###
twirl = simulation_opts['pauli_twirling']['twirl']     # if True, apply Pauli twirling
ntwirls = simulation_opts['pauli_twirling']['ntwirls'] # number of Pauli twirls

### dynamic decoupling ###
dd_method = simulation_opts['dynamic_decoupling']['method'] # if not none apply a given dynamic decoupling method

### Richardson ??? ###
richardson_level = simulation_opts['richardson_level'] # number of CNOTs for Richardson smearing

11 6


# Initialize IBM Account

In [53]:
### initialize IBM Quantum account ###

IBMQ.load_account()
hub = 'ibm-q-research'#account_info['hub']
group = 'fermilab-1'#account_info['group']
project = 'main'#account_info['project']
provider = IBMQ.get_provider(hub=hub, group=group, project=project)

IBMQAccountCredentialsNotFound: 'No IBM Quantum Experience credentials found.'

In [54]:
IBMQ.providers()

[]

# Choose target backend for simulations to run on

In [55]:
def get_backend(provider, machine, nqubits, simulator, sim_noise):
    
    ## Run on IBMQ qasm simulator
    if (simulator):
#         backend = provider.get_backend('ibmq_qasm_simulator')
        backend = Aer.get_backend('qasm_simulator')
        if (sim_noise != "None"):
            device_backend = provider.get_backend(sim_noise)
            noise_model = noise.NoiseModel.from_backend(device_backend)
            backend.set_options(noise_model=noise_model.to_dict())
            
        print(f'Running on ibmq_qasm_simulator with {sim_noise} noise_model\n') 
        return backend
    ## Run on specified quantum computer(s)
    else:
#         devices = provider.backends(filters=lambda x: not (x.name() == 'ibmq_casablanca' or x.name() == 'ibmq_armonk')
#                                     and nqubits <= x.configuration().n_qubits <= 7
#                                     and not x.configuration().simulator
#                                     and x.status().operational == True)
#         print(type(devices))
#         print(type(devices[0]))
        #print(f'Backends matching critera ares {devices}\n')
        #print(device_names)
        
        if (machine == 'least_busy'):
            machine = str(least_busy(devices))
            backend = provider.get_backend(machine)    
        
        elif (machine[0:4] == 'ibmq_'):
            backend = provider.get_backend(machine)   
            
        else:
            backend = provider.get_backend(str(machine))
            
        print(f'Running on machine {machine}\n')
        return backend
        

In [56]:

# machine
machine = backend_opts['machine']           # machine name or "least busy"
simulator = backend_opts['qasm_simulator']  # if True, use ibmq_qasm_simulator 
noise_model = backend_opts['noise_model']   # None or device noise model 
print(noise_model)
backend = get_backend(None, 'None', nqubits, True, 'None')

ibmq_belem
Running on ibmq_qasm_simulator with None noise_model



# Assign Runtime Program (and inputs)

assigns values from `yaml` file

In [57]:
# number of shots per simulation
nshots = runtime_opts['nshots'] 

# Not really sure what this is needed for...
program_id = runtime_opts['program_id']

# correct for readout error using calibration matrix if True
measurement_error_mitigation = runtime_opts['readout_error_mitigation'] 

#save results for each shot if True
memory = runtime_opts['memory'] 

# Degree of layout optimization by transpiler
# 1 is least optimized; 3 is most optimized
optimization_level = runtime_opts['optimization_level'] 

In [58]:
nsites2nqubits(nsites), nsites

(11, 6)

In [59]:
def build_circuit(nq, nsteps, epsilon, mass, init_state=None,
                  dynamic_decoupling=False, Twirling=False):
    
#     print(nq)
    if init_state is None:
        if nq == 3:
            qc = QuantumCircuit(nq, nq)
            qc.h(0)
            qc.cx(0, 1)
            qc.cx(0, 2)
            qc.x(2)
            qc.h(1)
#             qc = sprep.prepare_states_nsites2(nq)['meson']
        elif nq == 7:
            qc = QuantumCircuit(nq, nq)
            qc.h(2)
            qc.cx(2, 3)
            qc.cx(3, 4)
            qc.x([2, 6])
            qc.h([1, 3, 5])
        elif nq == 11:
            qc = QuantumCircuit(nq, nq)
            qc.h(4)
            qc.cx(4, 5)
            qc.cx(5, 6)
            qc.x([2, 6, 10])
            qc.h([1, 3, 5, 7, 9])
        elif nq == 15:
            qc = QuantumCircuit(nq, nq)
            qc.h(6)
            qc.cx(6, 7)
            qc.cx(7, 8)
            qc.x([2, 6, 10, 14])
            qc.h([1, 3, 5, 7, 9, 11, 13])
            #             qc = sprep.prepare_states_nsites4(nq)['meson']

        qc = Trot.trotter_evolution_generic(qc, nsites, epsilon, mass, nsteps)
#         for i in range(4):
        qc.h([i for i in range(0, nq, 2)])
        qc.measure([i for i in range(nq)], [i for i in range(nq)])
        return qc
#         return transpile(qc.decompose(), basis_gates=['cx', 'rz', 'sx'])
    else:
        print('not implemented')
        

# assemble circuits to run

In [60]:
circuits_no_dd = []
circuits_w_dd = []
tvals = []
twirl = False
for i in range(0, ntrotter + 1 + 380):
    print(i)
    tvals.append(np.round(i * epsilon, 8))
    # add time evolution circuit
    qc = build_circuit(nqubits, i, epsilon, mass)
    if twirl:
        qcs = pts.randomly_compile(qc, ncopy=ntwirls, backend=backend,
                                   initial_layout=initial_layout)
        qcs = transpile(qcs, basis_gates=['cx', 'rz', 'sx', 'sxdg'], optimization_level=optimization_level)
        circuits_no_dd += qcs
    else:
        circuits_no_dd.append(qc)
    # add rescaling circuit
    qc = build_circuit(nqubits, i, 0, mass)
    if twirl:
        qcs = pts.randomly_compile(qc, ncopy=ntwirls, backend=backend,
                                   initial_layout=initial_layout)
        qcs = transpile(qcs, basis_gates=['cx', 'rz', 'sx'], optimization_level=optimization_level)
        circuits_no_dd += qcs
    else:
        circuits_no_dd.append(qc)
    
circuits = circuits_no_dd

201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401


In [61]:
res = execute(circuits[::2], optimization_level=0, backend=Aer.get_backend('qasm_simulator'), shots=1e5)

In [63]:
obs = np.zeros(201)
for i in range(201):
    counts = res.result().get_counts(i)
    val = 0
    for key in counts.keys():
        if key[4:7].count('1') % 2 == 0:
            val += counts[key] / 1e5
        else:
            val -= counts[key] / 1e5
    obs[i] += val
#     for key in counts.keys():
#         if key[4:7].count('1') % 2 == 0:
#             val += counts[key] / 1e5
#         else:
#             val -= counts[key] / 1e5
#     obs[i] += val
obs

array([ 0.1266 ,  0.41052,  0.63122,  0.73814,  0.7116 ,  0.54826,
        0.29458,  0.001  , -0.26622, -0.45966, -0.54   , -0.48816,
       -0.30806, -0.0547 ,  0.22976,  0.4912 ,  0.6643 ,  0.71528,
        0.63842,  0.45388,  0.19114, -0.0807 , -0.32334, -0.46654,
       -0.49208, -0.40576, -0.20268,  0.05546,  0.32146,  0.53574,
        0.66294,  0.6685 ,  0.55662,  0.34816,  0.095  , -0.15268,
       -0.35274, -0.4468 , -0.43056, -0.30162, -0.08844,  0.16436,
        0.39708,  0.5638 ,  0.64264,  0.60062,  0.46062,  0.24452,
        0.01042, -0.21138, -0.36128, -0.40796, -0.355  , -0.19974,
        0.01174,  0.24102,  0.45118,  0.5795 ,  0.6079 ,  0.53918,
        0.38048,  0.16488, -0.04748, -0.23554, -0.35254, -0.36   ,
       -0.27978, -0.1111 ,  0.096  ,  0.3048 ,  0.4714 ,  0.56054,
        0.55416,  0.46412,  0.29314,  0.09222, -0.09784, -0.2487 ,
       -0.31462, -0.29506, -0.1853 , -0.0172 ,  0.17684,  0.35528,
        0.48408,  0.53094,  0.49076,  0.37858,  0.21642,  0.02

# build the runtime-object

In [35]:
program_inputs_list = []
for i in range(4):
    program_inputs = {'circuits': circuits[300 * i:300 * (i + 1)],
                      'optimization_level': 0,
                      'measurement_error_mitigation': measurement_error_mitigation,
                      'memory': memory,
                      'shots': nshots}
    program_inputs_list.append(program_inputs)

provider.runtime.run()

In [110]:

job1 = provider.runtime.run(program_id=program_id,
                           options={'backend_name': backend.name()},
                           inputs=program_inputs_list[0],
                           result_decoder=RunnerResult)

C:\Users\gusta\miniforge3\lib\site-packages\qiskit\providers\ibmq\runtime\utils.py:218: UserWarning: Callable <function <lambda> at 0x0000022599478040> is not JSON serializable and will be set to None.
  warnings.warn(f"Callable {obj} is not JSON serializable and will be set to None.")


In [ ]:

job2 = provider.runtime.run(program_id=program_id,
                           options={'backend_name': backend.name()},
                           inputs=program_inputs_list[1],
                           result_decoder=RunnerResult)

In [ ]:

job3 = provider.runtime.run(program_id=program_id,
                           options={'backend_name': backend.name()},
                           inputs=program_inputs_list[2],
                           result_decoder=RunnerResult)

In [ ]:

job4 = provider.runtime.run(program_id=program_id,
                           options={'backend_name': backend.name()},
                           inputs=program_inputs_list[3],
                           result_decoder=RunnerResult)

In [ ]:
jobs = [job1, job2, job3, job4]

In [ ]:
dataframe  = pandas.DataFrame(columns=['jobid', 'machine', 'date', 'sim type', 'nt', 'pauli twirling', 'dynamic decoupling', 'epsilon', 'mass', 'counts bare', 'counts mitigated'])
data_row = []
ntval = 0
for k in range(len(jobs)):
    for j in range(300):
        data_dict = {}
        data_dict['jobid'] = jobs[k].job_id()
        data_dict['machine'] = jobs[k].backend().name()
        data_dict['date'] = datetime.datetime.today()
        if (j // ntwirls) % 2 == 0:
            data_dict['sim type'] = 'evolution'
            data_dict['nt'] = ntval // (ntwirls * 2) + 1
            nt+=1
            data_dict['pauli twirling'] = twirl
            if k > 4:
                data_dict['dynamic decoupling'] = True
            else:
                data_dict['dynamic decoupling'] = False
            data_dict['epsilon'] = epsilon
            data_dict['mass'] = mass
            data_dict['counts bare'] = jobs[k].result().results[j].data.counts
            data_dict['counts mitigated'] = jobs[k].result().results[j].data.quasiprobabilities
            data_row.append(data_dict)
        else:
            data_dict['sim type'] = 'rescaling'
            data_dict['nt'] = ntval // (ntwirls * 2) + 1
            nt+=1
            data_dict['pauli twirling'] = twirl
            if k > 4:
                data_dict['dynamic decoupling'] = True
            else:
                data_dict['dynamic decoupling'] = False
            data_dict['epsilon'] = epsilon
            data_dict['mass'] = mass
            data_dict['counts bare'] = jobs[k].result().results[j].data.counts
            data_dict['counts mitigated'] = jobs[k].result().results[j].data.quasiprobabilities
            data_row.append(data_dict)
        
dataframe = pandas.DataFrame(data_row)

In [ ]:
dataframe.to_csv(f'simulation_production_run_on_{machine}_data={datetime.datetime.today().date()}_{nqubits}qubits_{jobs[0].job_id()}.csv')